In [1]:
import numpy as np
import cv2 
import os
from glob import glob
import matplotlib.pyplot as plt
from keras.applications import ResNet50
from keras.models import Model
import copy
import tensorflow as tf
from tensorflow import keras
import sys
import pandas
from keras_preprocessing.sequence import pad_sequences
import time
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
# from keras.layers import add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.layers import concatenate
from keras.layers import Reshape
from keras.layers import (LSTM, BatchNormalization, Dense, Dropout, Embedding,Input, Lambda, TimeDistributed)
from keras import backend as K
from keras import regularizers
from npy_append_array import NpyAppendArray
from tensorflow.keras.utils import to_categorical
import itertools
import pandas as pd

In [2]:
# images_path = 'Flickr_Data/Images/'
# images = glob(images_path+'*.jpg')
# captions_path = "results.csv"
captions_path = "results.csv"
# captions_path = "results.csv"
max_len = 40

In [3]:
# get sets train, test and dev

train = set()
f = open("Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt", "r")
for line in f:
    train.add(line.strip())
f.close()

test = set()
f = open("Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt", "r")
for line in f:
    test.add(line.strip())
f.close()

dev = set()
f = open("Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt", "r")
for line in f:
    dev.add(line.strip())

In [3]:
def resnet():
    resnet_model = ResNet50(include_top=True)
    resnet_model = Model(inputs=resnet_model.input, outputs=resnet_model.layers[-2].output)
    return resnet_model

In [4]:
# preprocess the images
# generate a dictionary of image filename -> feature vector
# start_index is inclusive

def generate_feature_vectors(num_of_images, images_path, model):
    img_feature_vectors = {}
    images = [f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f)) and f.endswith('.jpg')]
    
    count = 0
    for item in images[:num_of_images]:
        img = cv2.imread(images_path+item)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img.reshape(1, 224, 224, 3)

        feature_vector = model.predict(img, verbose=0).reshape(2048,)

        img_feature_vectors[item] = feature_vector

        count += 1

        if (count%50==0):
            print(count)

    return img_feature_vectors

In [5]:
# makes a string lowercase, prepends it with the string 'sos' and appends with 'eos'
def process_string(s):
    s = s.lower()
    s = 'sos ' + s + ' eos'
    return s

In [19]:

def generate_captions_dict(captions_path, img_feature_vectors):
    df = pd.read_csv(captions_path, delimiter='|')
    df.columns = ['image_name', 'comment_number', 'comment']
    del df['comment_number']
    df['comment'][19999] = ' A dog runs across the grass .'
    captions_dict = {}
    
    for index, row in df.iterrows():
        filename = row['image_name']
        caption = process_string(row['comment'])
        if filename in img_feature_vectors:
            if filename not in captions_dict:
                captions_dict[filename] = [caption]
            else:
                captions_dict[filename].append(caption)
    return captions_dict

In [7]:
# remove

def generate_captions_dict(captions_path, img_feature_vectors):
    captions_dict = {}

    f = open(captions_path, 'r').read().split('\n')

    # generate a dictionary of filenames to a list of captions
    for line in f:
        try:
            filename_caption = line.split('\t') 
            filename = filename_caption[0][:-2]
            caption = process_string(filename_caption[1])

            if filename in img_feature_vectors:
                if filename not in captions_dict:
                    captions_dict[filename] = [caption]
                else:
                    captions_dict[filename].append(caption)
        except:
            pass
        
    return captions_dict

In [20]:
def create_vocab(captions_dict, n, train):
    vocab_freq = {}
    vocab_dict = {}
    for key, value in captions_dict.items():
        if key not in train:
            continue
        for caption in value:
            caption_as_list = caption.split()
            for word in caption_as_list:
                if word not in vocab_freq:
                    vocab_freq[word] = 1
                else:
                    vocab_freq[word] = vocab_freq[word]+1
    words_to_keep = [w for w in vocab_freq if vocab_freq[w] >= n]
    
    count = 1
    for word in words_to_keep:
        if word not in vocab_dict:
            vocab_dict[word] = count
            count+=1
    return vocab_dict

In [9]:
def encode_string(s, vocab):
    s_list = s.split()
    encoded_string = []
    for word in s_list:
        if word in vocab:
            encoded_string.append(vocab[word])
    return encoded_string

In [10]:
def encode_captions_dict(captions_dict, vocab):
    captions_dict_encoded = copy.deepcopy(captions_dict)

    for filename, captions in captions_dict_encoded.items():
        for i, caption in enumerate(captions):
            captions[i] = encode_string(caption, vocab)
    return captions_dict_encoded

In [11]:
# batch size -> number of images will data be generated for at once

#hello
def generate_training_data(img_feature_vectors, captions_dict_encoded, max_len, vocab_len, batch_size, id_):
    
    X_file = f'X_{id_}.npy'
    y_in_file = f'y_in_{id_}.npy'
    y_out_file = f'y_out_{id_}.npy'
    filenames_file = f'filenames_{id_}.npy'
    
    X = []
    y_in = []
    y_out = []
    filenames = set()
    
    n = 0
    for filename, fv in img_feature_vectors.items():
        n+=1
        for caption in captions_dict_encoded[filename]:
            i = 0
            for word in caption:
                if i==0:
                    i+=1
                    continue
                y_in_item = [caption[:i]]
                y_in_item = pad_sequences(y_in_item, maxlen=max_len, truncating='post')[0]
                y_in.append(y_in_item)

                y_out_item = to_categorical([word], num_classes=vocab_len+1)[0]
                y_out.append(y_out_item)

                X.append(fv)
                
                filenames.add(filename)
                i+=1
        if n%batch_size==0 or n==len(img_feature_vectors):
            print(n)
            with NpyAppendArray(X_file) as npaa:
                npaa.append(np.array(X))
            with NpyAppendArray(y_in_file) as npaa:
                npaa.append(np.array(y_in))
            with NpyAppendArray(y_out_file) as npaa:
                npaa.append(np.array(y_out))
            X = []
            y_in = []
            y_out = []
            if n == len(img_feature_vectors):
                break
    np.save(filenames_file, filenames)

In [22]:
# resnet_model = resnet()
# img_feature_vectors = generate_feature_vectors(8091, images_path, resnet_model)
captions_dict = generate_captions_dict(captions_path, img_feature_vectors)
vocab = create_vocab(captions_dict, 10, train)
captions_dict_encoded = encode_captions_dict(captions_dict, vocab)

In [ ]:
images = [f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f)) and f.endswith('.jpg')]
images = set(images)

In [ ]:
def modify_filenames(filenames):
    modified_filenames = set()
    for filename in filenames:
        parts = filename.split("_")
        modified_filename = "".join(parts[:-1]) + "." + parts[-1].split(".")[-1]
        modified_filenames.add(modified_filename)
    return modified_filenames

In [ ]:
train = modify_filenames(train)
test = modify_filenames(test)
dev = modify_filenames(dev)
images = modify_filenames(images)

In [26]:
# np.save('30k_final/img_feature_vectors.npy', img_feature_vectors)
# np.save('30k_final/captions_dict.npy', captions_dict)
# np.save('30k_final/captions_dict_encoded.npy', captions_dict_encoded)
# np.save('30k_final/vocab_new.npy', vocab)

In [13]:
img_feature_vectors = np.load('30k/img_feature_vectors.npy', allow_pickle='TRUE').item()
# captions_dict_encoded = np.load('captions_dict_encoded.npy', allow_pickle='TRUE').item()
# vocab = np.load("vocab.npy", allow_pickle='TRUE').item()
# embedding_matrix = np.load('embedding_matrix.npy')

In [14]:
len(img_feature_vectors)

31783

In [15]:
images = list(img_feature_vectors.keys())
train = set(images[:25427])
dev = set(images[25427:28605])
test = set(images[28605:31783])

In [33]:
img_feature_vectors_new = {k: v for k, v in img_feature_vectors.items() if k in dev}

In [34]:
start_time = time.time()

vocab_len = len(vocab)

generate_training_data(img_feature_vectors_new, captions_dict_encoded, 40, vocab_len, 1000, "dev")

print(time.time()-start_time)

1000
2000
3000
3178
62.444525957107544


Word Embeddings and other stuff

In [ ]:
word_embeddings = {}
f = open("glove/glove.6B.200d.txt", encoding="utf-8")
for line in f:
    word_vector = line.split()
    word = word_vector[0]
    vector = np.asarray(word_vector[1:], dtype='float32')
    word_embeddings[word] = vector
f.close()

In [ ]:
vocab = np.load("vocab.npy", allow_pickle=True).item()

In [ ]:
word_embeddings.get("t-shirt")

In [ ]:
embedding_size = 200
vocab_size = 1965+1
# word to index

embedding_matrix = np.zeros((vocab_size, embedding_size))

for word, i in vocab.items():
    #if i < max_words:
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
np.save('embedding_matrix.npy', embedding_matrix)

In [ ]:
#version 1

embedding_size = 128
vocab_len = 3988+1
max_len = 40

#image
x = keras.Sequential([
    keras.layers.Input(shape=(2048,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.RepeatVector(max_len)
])

#caption
y = keras.Sequential([
    keras.layers.Embedding(vocab_len, embedding_size, input_length=max_len),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(embedding_size))
])

#concatinate outputs of y and x
z = keras.layers.Concatenate()([x.output, y.output])
z = keras.layers.LSTM(128, return_sequences=True)(z)
z = keras.layers.LSTM(512, return_sequences=False)(z)
z = keras.layers.Dense(vocab_len, activation='softmax')(z)

model = Model(inputs=[x.input, y.input], outputs=z)

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
#version 2

vocab_size = 1965+1
max_len = 40
embedding_size = 200

# image feature extractor model
image_input = Input(shape=(2048,))
ii1 = Dropout(0.5)(image_input)
ii2 = Dense(256, activation='relu')(ii1)

# partial caption sequence model
caption_input = Input(shape=(max_len,))
ci1 = Embedding(vocab_size, embedding_size, mask_zero=True)(caption_input)
ci2 = Dropout(0.5)(ci1)
ci3 = LSTM(256)(ci2)

# decoder (feed forward) model
output = add([ii2, ci3])
output = Dense(256, activation='relu')(output)
output = Dense(vocab_size, activation='softmax')(output)

# merge the two input models
model = Model(inputs=[image_input, caption_input ], outputs=output)

In [ ]:
import re
s = "string. With. hello-there Punctua%tion? hello' "
s = re.sub(r'[^\w\s\'\-]','',s)

In [ ]:
alist = ["a dog on a water slide", "a boy jumping at a tree", "a man sailing a boat"]

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1+1,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~')

In [ ]:
tokenizer.fit_on_texts(alist)

In [ ]:
tokenizer.word_index

In [ ]:
tokenizer.word_counts['a']

In [ ]:
# create a tokenizer which contains all words
# when encoding only use word if its frequency is greater than 10 